In [2]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI
import requests

In [3]:
response  =  requests.get('http://localhost:11434')
print(f'Status: ', response.content.decode())

Status:  Ollama is running


In [4]:
MODEL = 'llama3.2:1b'
BASE_URL = 'http://localhost:11434/v1'
ollama = OpenAI(base_url=BASE_URL,api_key='ollama')

In [15]:
!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 74701a8c35f6: 100% ▕██████████████████▏ 1.3 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 4f659a1e86d7: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [8]:
links  = fetch_website_links('http://quotes.toscrape.com')
links

['/',
 '/login',
 '/author/Albert-Einstein',
 '/tag/change/page/1/',
 '/tag/deep-thoughts/page/1/',
 '/tag/thinking/page/1/',
 '/tag/world/page/1/',
 '/author/J-K-Rowling',
 '/tag/abilities/page/1/',
 '/tag/choices/page/1/',
 '/author/Albert-Einstein',
 '/tag/inspirational/page/1/',
 '/tag/life/page/1/',
 '/tag/live/page/1/',
 '/tag/miracle/page/1/',
 '/tag/miracles/page/1/',
 '/author/Jane-Austen',
 '/tag/aliteracy/page/1/',
 '/tag/books/page/1/',
 '/tag/classic/page/1/',
 '/tag/humor/page/1/',
 '/author/Marilyn-Monroe',
 '/tag/be-yourself/page/1/',
 '/tag/inspirational/page/1/',
 '/author/Albert-Einstein',
 '/tag/adulthood/page/1/',
 '/tag/success/page/1/',
 '/tag/value/page/1/',
 '/author/Andre-Gide',
 '/tag/life/page/1/',
 '/tag/love/page/1/',
 '/author/Thomas-A-Edison',
 '/tag/edison/page/1/',
 '/tag/failure/page/1/',
 '/tag/inspirational/page/1/',
 '/tag/paraphrased/page/1/',
 '/author/Eleanor-Roosevelt',
 '/tag/misattributed-eleanor-roosevelt/page/1/',
 '/author/Steve-Martin',
 

In [9]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}"""

In [10]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [11]:
print(get_links_user_prompt("http://quotes.toscrape.com"))


Here is the list of links on the website http://quotes.toscrape.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

/
/login
/author/Albert-Einstein
/tag/change/page/1/
/tag/deep-thoughts/page/1/
/tag/thinking/page/1/
/tag/world/page/1/
/author/J-K-Rowling
/tag/abilities/page/1/
/tag/choices/page/1/
/author/Albert-Einstein
/tag/inspirational/page/1/
/tag/life/page/1/
/tag/live/page/1/
/tag/miracle/page/1/
/tag/miracles/page/1/
/author/Jane-Austen
/tag/aliteracy/page/1/
/tag/books/page/1/
/tag/classic/page/1/
/tag/humor/page/1/
/author/Marilyn-Monroe
/tag/be-yourself/page/1/
/tag/inspirational/page/1/
/author/Albert-Einstein
/tag/adulthood/page/1/
/tag/success/page/1/
/tag/value/page/1/
/author/Andre-Gide
/tag/life/page/1/
/tag/love/page/1/
/author/Thomas-A-Edison
/tag/edison/page/1/
/tag/failure/page/1/


In [19]:
def select_relevant_links(url):
    user_prompt = get_links_user_prompt(url)
    messages = [
        {'role': 'system', 'content': link_system_prompt},
        {'role': 'user',   'content': user_prompt}
    ]
    response = ollama.chat.completions.create(model=MODEL,
                                              messages=messages,
                                              response_format={'type':'json_object'})
    result = response.choices[0].message.content

    if isinstance(result, str):
        links = json.loads(result)
    else:
        links = result
    return links

In [20]:
select_relevant_links('http://quotes.toscrape.com')

{'links': [{'type': 'about page', 'url': 'https://full.url/goes/here/about'},
  {'type': 'careers page', 'url': 'https://another.full.url/careers'},
  {'type': 'tag page', 'url': 'https://quotes.toscrape.com/tag/page/1/'}]}

In [21]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [22]:
print(fetch_page_and_all_relevant_links("http://quotes.toscrape.com"))

## Landing Page:

Quotes to Scrape

Quotes to Scrape
Login
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
by
Albert Einstein
(about)
Tags:
change
deep-thoughts
thinking
world
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
by
J.K. Rowling
(about)
Tags:
abilities
choices
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
by
Albert Einstein
(about)
Tags:
inspirational
life
live
miracle
miracles
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
by
Jane Austen
(about)
Tags:
aliteracy
books
classic
humor
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
by
Marilyn Monroe
(about)
Tags:
be-yourself
inspirational
“Try not to become a man of success. Rather become a man of value.”
by
Albert Eins

In [23]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000]
    return user_prompt

In [25]:
def stream_brochure(company_name, url):
    stream = ollama.chat.completions.create(
        model="llama3.2:1b",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [26]:
stream_brochure(company_name='quotes to scrape',url="http://quotes.toscrape.com")

# The Quotes to Scrape Story

## A Tale of Code and Curiosity

In a world where words come alive in code, **Quotes to Scrape** takes on the responsibility of curating the most fascinating quotes for your eyes. As a company, we believe that the power of language lies in its ability to inspire and educate. Our mission is to bring you quotes from around the globe, in all their depths and complexities.

## Who We Are

At Quotes to Scrape, we are a team of wordsmiths dedicated to unlocking the secret meanings behind great quotes. Our goal is to create an unparalleled collection that will leave you thinking deep and pondering life's mysteries. Whether it's the musings of Albert Einstein, Jane Austen, or Marilyn Monroe, our curated quotes will guide you down the rabbit hole of philosophical discussion.

## What We Offer

•   **The Best Quotes Around the World**: Our archive boasts a diverse collection of quotes from renowned sources, including GoodReads, with over 10 tagged categories.
•   **Your Favorite Lists**: Browse through our curated "Top Ten" lists on love, inspirational life, and humor to discover new favorites among friends and bookworms alike.

## The Quotes

Some of the quotes that inspire us include:

*   “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
    by Albert Einstein
        (about)
            Tags:
                change
                deep-thoughts
                thinking
                world
*   “Try not to become a man of success. Rather become a man of value.”
    by Albert Einstein
            (about)
                Tags:
                    adulthood
                    success
                    value
*   “I have not failed. I've just found 10,000 ways that won't work.”
    by Thomas A. Edison
        (about)
            Tags:
                failure
                inspirational
                paraphrased
            – Misattributed to Marilyn Monroe

## Join the Conversation

Connect with us on various platforms:

*   **GoodReads**: Discover and share quotes from fellow book lovers.
*   **Books & Co.**: Stay updated on new additions to our archives.

Join the conversation, explore our collection, and remember to "Be yourself; everyone else is already taken."